In [401]:
# bs4불러오기
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from tqdm import tqdm
import random

In [410]:
user_agent = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"}

def recipe_link_crawling(start_page,end_page):    
    recipe_link = []
    failed_pages = []
    for i in tqdm(range(start_page,end_page)):
        time.sleep(random.uniform(0.5,1))
        for j in range(3):
            try:
                res = requests.get(f"https://www.10000recipe.com/recipe/list.html?order=reco&page={i}",user_agent)
                if res.status_code != 200:
                    print(f"{i}페이지 요청 실패: {res.status_code}")
                    continue
                soup = BeautifulSoup(res.text,"html.parser")
                links = soup.find_all("a", class_="common_sp_link")
                recipe_link += [i['href'].split('/')[-1] for i in links]
                break
            except requests.exceptions.RequestException as e:
                print(f"{i}페이지 에러 발생: {e}, 재시도 중 ({j+1}/3)")
                time.sleep(random.uniform(1, 2))
        else:
            failed_pages.append(i)
    return recipe_link, failed_pages

all_page_link,failed_page_link = recipe_link_crawling(1,5861)


 48%|████▊     | 2817/5860 [50:32<58:04,  1.14s/it]  

2818페이지 에러 발생: HTTPSConnectionPool(host='www.10000recipe.com', port=443): Max retries exceeded with url: /recipe/list.html?order=reco&page=2818&User-Agent=Mozilla%2F5.0+%28Windows+NT+10.0%3B+Win64%3B+x64%29+AppleWebKit%2F537.36+%28KHTML%2C+like+Gecko%29+Chrome%2F131.0.0.0+Safari%2F537.36 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000021F84130DF0>, 'Connection to www.10000recipe.com timed out. (connect timeout=None)')), 재시도 중 (1/3)


 81%|████████  | 4726/5860 [1:25:36<22:08,  1.17s/it]

4727페이지 에러 발생: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), 재시도 중 (1/3)


100%|██████████| 5860/5860 [1:46:04<00:00,  1.09s/it]


In [470]:
def safe_find(soup, selector, attr=None, multiple=False):
    try:
        if multiple:
            elements = soup.select(selector)
            return [e.text.strip() for e in elements] if elements else None
        element = soup.select_one(selector)
        return element.text.strip() if element else None
    except Exception as e:
        print(f"Error extracting {selector}: {e}")
        return None
    
def fetch_page(url, retries=3, delay=2):
    for attempt in range(retries):
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response
        except requests.exceptions.RequestException as e:
            print(f"Error fetching {url} (attempt {attempt+1}): {e}")
            time.sleep(delay)
    return None

def recipe_crawler(num_list):
    title, view, serving, cooking_time, level, ingredients = [], [], [], [], [], []

    for i in tqdm(num_list):
        res = fetch_page(f"https://www.10000recipe.com/recipe/{i}")
        if res is None:
            print(f"Skipping recipe {i} due to repeated errors.")
            title.append(None)
            view.append(None)
            serving.append(None)
            cooking_time.append(None)
            level.append(None)
            ingredients.append(None)
            continue

        soup = BeautifulSoup(res.text, "html.parser")
        
        title.append(safe_find(soup, "div.view2_summary.st3 h3"))
        view.append(safe_find(soup, "span.hit.font_num"))
        serving.append(safe_find(soup, "span.view2_summary_info1"))
        cooking_time.append(safe_find(soup, "span.view2_summary_info2"))
        level.append(safe_find(soup, "span.view2_summary_info3"))
        
        ingredient_data = []
        for name, qty in zip(
            soup.select("#divConfirmedMaterialArea .ingre_list_name"),
            soup.select("#divConfirmedMaterialArea .ingre_list_ea")
        ):
            ingredient_data.append({"name": name.text.strip().split()[0], "quantity": qty.text.strip()})
        ingredients.append(ingredient_data if ingredient_data else None)
        
        time.sleep(random.uniform(0.5, 1))

    return title, view, serving, cooking_time, level, ingredients

In [ ]:
# with open('data.pickle','wb') as fw:
#     pickle.dump(all_page_link, fw)

import pickle
with open("data.pickle","rb") as fr:
    data = pickle.load(fr)
print(len(data))

234400


In [ ]:
title, view, serving, cooking_time, level, ingredients = recipe_crawler(data)

In [ ]:
recipe_df = pd.DataFrame(
    {"레시피번호":,
    "제목":title,
    "조회수":view,
    "식사량":serving,
    "조리시간":cooking_time,
    "난이도":level,
    "재료":[[item['name']for item in recipe] for recipe in ingredients],
    "재료의양":[[item['quantity']for item in recipe] for recipe in ingredients]})

In [482]:
recipe_df

,레시피번호,제목,조회수,식사량,조리시간,난이도,재료,재료의양
0,7039682,[가지양념구이] 간단하지만 맛있는 황금레시피 | 신혼밥상,154,2인분,30분 이내,초급,"[가지, 물, 밀가루, 진간장, 식초, 다진마늘, 꿀, 물, 청양고추]","[2개, 60ml, 2T, 2T, 1T, 1/2T, 2T, 3T, 조금]"
1,7039214,치킨너겟 교촌 허니콤보 만들기,545,1인분,15분 이내,초급,"[치킨너겟, 진간장, 맛술, 물, 다진마늘, 꿀, 굴소스]","[11개, 1T, 1T, 1T, 0.3T, 1T, 0.5T]"
2,6984517,도시락 반찬으로 최고! 스팸감자조림♡,"72,882",2인분,10분 이내,초급,"[감자, 스팸, 진간장, 설탕, 후추, 올리고당, 다진마늘, 물]","[2개, 200g, 4스푼, 4스푼, 조금, 1스푼, 1스푼, 120ml]"
